In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [4]:
data = pd.read_csv("../data/merged_data.csv")
data = data.drop("Unnamed: 0", axis=1)

In [5]:
data

,city,year,geomobility,housing_price,income,median_age,unemployment,truck_cost
0,"Albuquerque, NM",2005,482442,153134.096654,41820,35.8,14.533702,2110.947613
1,"Albuquerque, NM",2006,485982,170259.007619,43021,35.3,18.786019,2183.914452
2,"Albuquerque, NM",2007,505490,193305.919721,43677,35.2,19.911130,2136.401161
3,"Albuquerque, NM",2008,506706,200215.211429,46437,35.5,19.150972,1914.106839
4,"Albuquerque, NM",2009,519532,187486.929321,44594,34.0,13.877965,1568.787032
...,...,...,...,...,...,...,...,...
622,"Wichita, KS",2019,384278,134578.073415,55056,35.3,5.507219,1875.071344
623,"Wichita, KS",2020,385454,144698.977594,53466,35.3,5.570981,1730.099813
624,"Wichita, KS",2021,390384,158485.933434,56293,35.6,7.203338,1899.764297
625,"Wichita, KS",2022,393547,177256.373997,59277,36.2,6.025616,1916.491781


Basic Regression, no scaling

In [15]:

X_first = data[['city', 'year']]
Y_first = data[['income', 'median_age', 'unemployment', 'truck_cost', 'geomobility']]

column_transformer = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(drop='first'), ['city'])  # One-hot encoding for 'city'
    ],
    remainder='passthrough'  # Keep the remaining columns (year)
)

# Transform the features
X_first_transformed = column_transformer.fit_transform(X_first)

# Split the data into training and testing sets
X_train_first, X_test_first, Y_train_first, Y_test_first = train_test_split(X_first_transformed, Y_first, test_size=0.2, random_state=42)


# Train the model
model_first = LinearRegression()
model_first.fit(X_train_first, Y_train_first)

# Predict on the test set
Y_pred_first = model_first.predict(X_test_first)

# Calculate the mean squared error
mse_first = mean_squared_error(Y_test_first, Y_pred_first)
print(f"Mean Squared Error for First Regression: {mse_first}")

# Prepare future data for prediction
future_data = pd.DataFrame({
    'city': ['Baltimore, MD'],  # Change to the desired city
    'year': [2025]
})

# Transform the future data to match training set
future_data_transformed = column_transformer.transform(future_data)

# Make predictions for the future year
future_predictions = model_first.predict(future_data_transformed)
predicted_income, predicted_mean_age, predicted_unemployment, predicted_truck_price, predicted_geomobility = future_predictions[0]
print(f"Predicted Values for Baltimore 2025: Income={predicted_income}, Mean Age={predicted_mean_age}, Unemployment={predicted_unemployment}, Truck Price={predicted_truck_price}, Geomobility={predicted_geomobility}")


Mean Squared Error for First Regression: 356969008.75634706
Predicted Values for Baltimore 2025: Income=62037.01126759127, Mean Age=35.97949050221146, Unemployment=3.9456085013288202, Truck Price=2056.42043627804, Geomobility=663345.4991583321


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers

# Load your data
# Assuming your data is already loaded into a DataFrame called 'data'

# Prepare features and targets
X_first = data[['city', 'year']]
Y_first = data[['income', 'median_age', 'unemployment', 'truck_cost', 'geomobility']]

# One-hot encoding for 'city' with dense output
column_transformer = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(drop='first', sparse_output=False), ['city'])  # Use sparse_output=False
    ],
    remainder='passthrough'  # Keep the remaining columns (year)
)

# Transform the features
X_first_transformed = column_transformer.fit_transform(X_first)

# Normalize the input features
scaler_X = MinMaxScaler()
X_first_normalized = scaler_X.fit_transform(X_first_transformed)

# Normalize the target variables
scaler_Y = MinMaxScaler()
Y_first_normalized = scaler_Y.fit_transform(Y_first)

# Reshape the input data for RNN
X_first_normalized = np.array(X_first_normalized).reshape(X_first_normalized.shape[0], 1, X_first_normalized.shape[1])

# Split the data into training and testing sets
X_train_first, X_test_first, Y_train_first, Y_test_first = train_test_split(X_first_normalized, Y_first_normalized, test_size=0.2, random_state=42)

# Build a more complex RNN model
model_first = keras.Sequential([
    layers.LSTM(100, activation='relu', return_sequences=True, input_shape=(X_train_first.shape[1], X_train_first.shape[2])),
    layers.LSTM(50, activation='relu'),
    layers.Dense(5)  # Output layer for 5 target variables
])

# Compile the model
model_first.compile(optimizer='adam', loss='mse')

# Train the model with more epochs
model_first.fit(X_train_first, Y_train_first, epochs=200, validation_split=0.2, batch_size=32)

# Predict on the test set
Y_pred_first = model_first.predict(X_test_first)

# Inverse transform to get original scale predictions
Y_pred_first_inverse = scaler_Y.inverse_transform(Y_pred_first)

# Calculate the mean squared error
mse_first = mean_squared_error(Y_test_first, Y_pred_first)
print(f"Mean Squared Error for First Regression: {mse_first}")

# Prepare future data for prediction
future_data = pd.DataFrame({
    'city': ['Baltimore, MD'],  # Change to the desired city
    'year': [2025]
})

# Transform the future data to match training set
future_data_transformed = column_transformer.transform(future_data)

# Normalize future data
future_data_transformed_normalized = scaler_X.transform(future_data_transformed)

# Reshape the future data for RNN
future_data_transformed_normalized = np.array(future_data_transformed_normalized).reshape(-1, 1, future_data_transformed_normalized.shape[1])

# Make predictions for the future year
future_predictions = model_first.predict(future_data_transformed_normalized)
future_predictions_inverse = scaler_Y.inverse_transform(future_predictions)

# Extract predicted values
predicted_income, predicted_mean_age, predicted_unemployment, predicted_truck_price, predicted_geomobility = future_predictions_inverse[0]
print(f"Predicted Values for Baltimore 2025: Income={predicted_income}, Mean Age={predicted_mean_age}, Unemployment={predicted_unemployment}, Truck Price={predicted_truck_price}, Geomobility={predicted_geomobility}")


Epoch 1/200


/Users/achyutannarayanan/Desktop/College/HackGT/Realty_Pulse/djangoProject/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1567 - val_loss: 0.1348
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1327 - val_loss: 0.1136
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1099 - val_loss: 0.0847
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0796 - val_loss: 0.0509
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0494 - val_loss: 0.0336
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0321 - val_loss: 0.0276
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0290 - val_loss: 0.0251
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0263 - val_loss: 0.0237
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0241 - val_loss: 0.0226
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0235 - val_loss: 0.0214
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0204 - val_loss: 0.0200
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0208 - val_l

In [32]:
future_data = pd.DataFrame({
    'city': ['Atlanta, GA'],  # Change to the desired city
    'year': [2027]
})

# Transform the future data to match training set
future_data_transformed = column_transformer.transform(future_data)

# Normalize future data
future_data_transformed_normalized = scaler_X.transform(future_data_transformed)

# Reshape the future data for RNN
future_data_transformed_normalized = np.array(future_data_transformed_normalized).reshape(-1, 1, future_data_transformed_normalized.shape[1])

# Make predictions for the future year
future_predictions = model_first.predict(future_data_transformed_normalized)
future_predictions_inverse = scaler_Y.inverse_transform(future_predictions)

# Extract predicted values
predicted_income, predicted_mean_age, predicted_unemployment, predicted_truck_price, predicted_geomobility = future_predictions_inverse[0]
print(f"Predicted Values for Baltimore 2025: Income={predicted_income}, Mean Age={predicted_mean_age}, Unemployment={predicted_unemployment}, Truck Price={predicted_truck_price}, Geomobility={predicted_geomobility}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Predicted Values for Baltimore 2025: Income=102241.6953125, Mean Age=35.54972457885742, Unemployment=2.7256431579589844, Truck Price=1919.6668701171875, Geomobility=502817.5625


In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers

# Prepare the dataset for the second regression
X_second = data[['city', 'year', 'income', 'median_age', 'unemployment', 'truck_cost', 'geomobility']]
Y_second = data['housing_price']

# Transform the features for the second regression
column_transformer_second = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(drop='first', sparse_output=False), ['city'])  # Use sparse_output=False
    ],
    remainder='passthrough'  # Keep the remaining columns (year, income, etc.)
)

# Transform the features
X_second_transformed = column_transformer_second.fit_transform(X_second)

# Normalize the input features
scaler_X_second = MinMaxScaler()
X_second_normalized = scaler_X_second.fit_transform(X_second_transformed)

# Normalize the target variable (housing price)
scaler_Y_second = MinMaxScaler()
Y_second_normalized = scaler_Y_second.fit_transform(Y_second.values.reshape(-1, 1))

# Reshape the input data for RNN
X_second_normalized = np.array(X_second_normalized).reshape(X_second_normalized.shape[0], 1, X_second_normalized.shape[1])

# Split the data into training and testing sets
X_train_second, X_test_second, Y_train_second, Y_test_second = train_test_split(X_second_normalized, Y_second_normalized, test_size=0.2, random_state=42)

# Build an RNN model for housing price prediction
model_second = keras.Sequential([
    layers.LSTM(100, activation='relu', return_sequences=True, input_shape=(X_train_second.shape[1], X_train_second.shape[2])),
    layers.LSTM(50, activation='relu'),
    layers.Dense(1)  # Output layer for housing price
])

# Compile the model
model_second.compile(optimizer='adam', loss='mse')

# Train the model
model_second.fit(X_train_second, Y_train_second, epochs=200, validation_split=0.2, batch_size=32)

# Predict on the test set
Y_pred_second = model_second.predict(X_test_second)

# Inverse transform to get original scale predictions
Y_pred_second_inverse = scaler_Y_second.inverse_transform(Y_pred_second)

# Calculate the mean squared error
mse_second = mean_squared_error(Y_test_second, Y_pred_second)
print(f"Mean Squared Error for Second Regression: {mse_second}")

# Prepare future data for housing price prediction using predicted values from first regression
future_housing_data = pd.DataFrame({
    'city': ['Baltimore, MD'],  # Change to the desired city
    'year': [2025],
    'income': [predicted_income],
    'median_age': [predicted_mean_age],
    'unemployment': [predicted_unemployment],
    'truck_cost': [predicted_truck_price],  # Ensure the correct name is used
    'geomobility': [predicted_geomobility]
})

# Transform the future housing data to match training set
future_housing_data_transformed = column_transformer_second.transform(future_housing_data)

# Normalize the future housing data
future_housing_data_transformed_normalized = scaler_X_second.transform(future_housing_data_transformed)

# Reshape the future data for RNN
future_housing_data_transformed_normalized = np.array(future_housing_data_transformed_normalized).reshape(-1, 1, future_housing_data_transformed_normalized.shape[1])

# Make predictions for housing price
predicted_housing_price = model_second.predict(future_housing_data_transformed_normalized)
predicted_housing_price_inverse = scaler_Y_second.inverse_transform(predicted_housing_price)

print(f"Predicted Housing Price for Baltimore 2025: {predicted_housing_price_inverse[0][0]}")


Epoch 1/200


/Users/achyutannarayanan/Desktop/College/HackGT/Realty_Pulse/djangoProject/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0817 - val_loss: 0.0469
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0528 - val_loss: 0.0243
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0255 - val_loss: 0.0152
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0179 - val_loss: 0.0136
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0153 - val_loss: 0.0088
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0100 - val_loss: 0.0067
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0063 - val_loss: 0.0050
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0043 - val_loss: 0.0039
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0040 - val_loss: 0.0035
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0033 - val_loss: 0.0034
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0031 - val_loss: 0.0033
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0035 - val_l

In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers

# Load your data
# Assuming your data is already loaded into a DataFrame called 'data'

def predict_city_data(city, year):
    # Prepare features and targets for the first regression
    X_first = data[['city', 'year']]
    Y_first = data[['income', 'median_age', 'unemployment', 'truck_cost', 'geomobility']]

    # One-hot encoding for 'city'
    column_transformer_first = ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(drop='first', sparse_output=False), ['city'])
        ],
        remainder='passthrough'  # Keep the remaining columns (year)
    )

    # Transform and normalize the features and targets for the first regression
    X_first_transformed = column_transformer_first.fit_transform(X_first)
    scaler_X_first = MinMaxScaler()
    X_first_normalized = scaler_X_first.fit_transform(X_first_transformed)
    scaler_Y_first = MinMaxScaler()
    Y_first_normalized = scaler_Y_first.fit_transform(Y_first)

    # Reshape the input data for RNN
    X_first_normalized = np.array(X_first_normalized).reshape(X_first_normalized.shape[0], 1, X_first_normalized.shape[1])

    # Split the data into training and testing sets
    X_train_first, X_test_first, Y_train_first, Y_test_first = train_test_split(X_first_normalized, Y_first_normalized, test_size=0.2, random_state=42)

    # Build and train the RNN model for socio-economic indicators
    model_first = keras.Sequential([
        layers.LSTM(100, activation='relu', return_sequences=True, input_shape=(X_train_first.shape[1], X_train_first.shape[2])),
        layers.LSTM(50, activation='relu'),
        layers.Dense(5)  # Output layer for 5 target variables
    ])

    model_first.compile(optimizer='adam', loss='mse')
    model_first.fit(X_train_first, Y_train_first, epochs=300, validation_split=0.2, batch_size=32)

    # Prepare future data for socio-economic prediction
    future_data = pd.DataFrame({
        'city': [city],
        'year': [year]
    })

    # Transform and normalize the future data
    future_data_transformed = column_transformer_first.transform(future_data)
    future_data_transformed_normalized = scaler_X_first.transform(future_data_transformed)
    future_data_transformed_normalized = np.array(future_data_transformed_normalized).reshape(-1, 1, future_data_transformed_normalized.shape[1])

    # Make predictions for the future year
    future_predictions = model_first.predict(future_data_transformed_normalized)
    future_predictions_inverse = scaler_Y_first.inverse_transform(future_predictions)

    # Extract predicted values
    predicted_income, predicted_mean_age, predicted_unemployment, predicted_truck_price, predicted_geomobility = future_predictions_inverse[0]

    print(f"Predicted Values for {city} {year}: Income={predicted_income}, Mean Age={predicted_mean_age}, Unemployment={predicted_unemployment}, Truck Price={predicted_truck_price}, Geomobility={predicted_geomobility}")

    # Prepare the dataset for the second regression
    X_second = data[['city', 'year', 'income', 'median_age', 'unemployment', 'truck_cost', 'geomobility']]
    Y_second = data['housing_price']

    # Transform the features for the second regression
    column_transformer_second = ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(drop='first', sparse_output=False), ['city'])
        ],
        remainder='passthrough'  # Keep the remaining columns (year, income, etc.)
    )

    # Transform and normalize the features and target for the second regression
    X_second_transformed = column_transformer_second.fit_transform(X_second)
    scaler_X_second = MinMaxScaler()
    X_second_normalized = scaler_X_second.fit_transform(X_second_transformed)
    scaler_Y_second = MinMaxScaler()
    Y_second_normalized = scaler_Y_second.fit_transform(Y_second.values.reshape(-1, 1))

    # Reshape the input data for RNN
    X_second_normalized = np.array(X_second_normalized).reshape(X_second_normalized.shape[0], 1, X_second_normalized.shape[1])

    # Split the data into training and testing sets
    X_train_second, X_test_second, Y_train_second, Y_test_second = train_test_split(X_second_normalized, Y_second_normalized, test_size=0.2, random_state=42)

    # Build and train the RNN model for housing price prediction
    model_second = keras.Sequential([
        layers.LSTM(100, activation='relu', return_sequences=True, input_shape=(X_train_second.shape[1], X_train_second.shape[2])),
        layers.LSTM(50, activation='relu'),
        layers.Dense(1)  # Output layer for housing price
    ])

    model_second.compile(optimizer='adam', loss='mse')
    model_second.fit(X_train_second, Y_train_second, epochs=300, validation_split=0.2, batch_size=32)

    # Prepare future data for housing price prediction
    future_housing_data = pd.DataFrame({
        'city': [city],
        'year': [year],
        'income': [predicted_income],
        'median_age': [predicted_mean_age],
        'unemployment': [predicted_unemployment],
        'truck_cost': [predicted_truck_price],
        'geomobility': [predicted_geomobility]
    })

    # Transform and normalize the future housing data
    future_housing_data_transformed = column_transformer_second.transform(future_housing_data)
    future_housing_data_transformed_normalized = scaler_X_second.transform(future_housing_data_transformed)
    future_housing_data_transformed_normalized = np.array(future_housing_data_transformed_normalized).reshape(-1, 1, future_housing_data_transformed_normalized.shape[1])

    # Make predictions for housing price
    predicted_housing_price = model_second.predict(future_housing_data_transformed_normalized)
    predicted_housing_price_inverse = scaler_Y_second.inverse_transform(predicted_housing_price)

    print(f"Predicted Housing Price for {city} {year}: {predicted_housing_price_inverse[0][0]}")

# Example usage
predict_city_data('Baltimore, MD', 2025)


Epoch 1/300


/Users/achyutannarayanan/Desktop/College/HackGT/Realty_Pulse/djangoProject/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1583 - val_loss: 0.1288
Epoch 2/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1256 - val_loss: 0.0988
Epoch 3/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0943 - val_loss: 0.0608
Epoch 4/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0531 - val_loss: 0.0346
Epoch 5/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0361 - val_loss: 0.0297
Epoch 6/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0301 - val_loss: 0.0259
Epoch 7/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0275 - val_loss: 0.0244
Epoch 8/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0251 - val_loss: 0.0234
Epoch 9/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0251 - val_loss: 0.0222
Epoch 10/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0233 - val_loss: 0.0208
Epoch 11/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0210 - val_loss: 0.0192
Epoch 12/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0207 - val_l

/Users/achyutannarayanan/Desktop/College/HackGT/Realty_Pulse/djangoProject/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0951 - val_loss: 0.0492
Epoch 2/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0634 - val_loss: 0.0255
Epoch 3/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0363 - val_loss: 0.0145
Epoch 4/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0181 - val_loss: 0.0133
Epoch 5/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0145 - val_loss: 0.0088
Epoch 6/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0126 - val_loss: 0.0067
Epoch 7/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0073 - val_loss: 0.0050
Epoch 8/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0054 - val_loss: 0.0038
Epoch 9/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0040 - val_loss: 0.0033
Epoch 10/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0035 - val_loss: 0.0031
Epoch 11/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0031 - val_loss: 0.0032
Epoch 12/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0033 - val_l

In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow import keras
from tensorflow.keras import layers

# Load your data
# Assuming your data is already loaded into a DataFrame called 'data'

# Function to train the models and store them
def train_models(data):
    # Prepare features and targets for the first regression
    X_first = data[['city', 'year']]
    Y_first = data[['income', 'median_age', 'unemployment', 'truck_cost', 'geomobility']]

    # One-hot encoding for 'city'
    column_transformer_first = ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(drop='first', sparse_output=False), ['city'])
        ],
        remainder='passthrough'  # Keep the remaining columns (year)
    )

    # Transform and normalize the features and targets for the first regression
    X_first_transformed = column_transformer_first.fit_transform(X_first)
    scaler_X_first = MinMaxScaler()
    X_first_normalized = scaler_X_first.fit_transform(X_first_transformed)
    scaler_Y_first = MinMaxScaler()
    Y_first_normalized = scaler_Y_first.fit_transform(Y_first)

    # Reshape the input data for RNN
    X_first_normalized = np.array(X_first_normalized).reshape(X_first_normalized.shape[0], 1, X_first_normalized.shape[1])

    # Build and train the RNN model for socio-economic indicators
    model_first = keras.Sequential([
        layers.LSTM(100, activation='relu', return_sequences=True, input_shape=(X_first_normalized.shape[1], X_first_normalized.shape[2])),
        layers.LSTM(50, activation='relu'),
        layers.Dense(5)  # Output layer for 5 target variables
    ])

    model_first.compile(optimizer='adam', loss='mse')
    model_first.fit(X_first_normalized, Y_first_normalized, epochs=300, validation_split=0.2, batch_size=32)

    # Prepare the dataset for the second regression
    X_second = data[['city', 'year', 'income', 'median_age', 'unemployment', 'truck_cost', 'geomobility']]
    Y_second = data['housing_price']

    # Transform the features for the second regression
    column_transformer_second = ColumnTransformer(
        transformers=[
            ('encoder', OneHotEncoder(drop='first', sparse_output=False), ['city'])
        ],
        remainder='passthrough'  # Keep the remaining columns (year, income, etc.)
    )

    # Transform and normalize the features and target for the second regression
    X_second_transformed = column_transformer_second.fit_transform(X_second)
    scaler_X_second = MinMaxScaler()
    X_second_normalized = scaler_X_second.fit_transform(X_second_transformed)
    scaler_Y_second = MinMaxScaler()
    Y_second_normalized = scaler_Y_second.fit_transform(Y_second.values.reshape(-1, 1))

    # Reshape the input data for RNN
    X_second_normalized = np.array(X_second_normalized).reshape(X_second_normalized.shape[0], 1, X_second_normalized.shape[1])

    # Build and train the RNN model for housing price prediction
    model_second = keras.Sequential([
        layers.LSTM(100, activation='relu', return_sequences=True, input_shape=(X_second_normalized.shape[1], X_second_normalized.shape[2])),
        layers.LSTM(50, activation='relu'),
        layers.Dense(1)  # Output layer for housing price
    ])

    model_second.compile(optimizer='adam', loss='mse')
    model_second.fit(X_second_normalized, Y_second_normalized, epochs=300, validation_split=0.2, batch_size=32)

    return model_first, scaler_X_first, scaler_Y_first, column_transformer_first, model_second, scaler_X_second, scaler_Y_second, column_transformer_second

# Function to make predictions for multiple cities and years using pre-trained models
def predict_city_data(cities, years, model_first, scaler_X_first, scaler_Y_first, column_transformer_first, model_second, scaler_X_second, scaler_Y_second, column_transformer_second):
    results = []  # List to store results

    for city in cities:
        for year in years:
            print(f"\nPredicting for {city} in {year}...")

            # Prepare future data for socio-economic prediction
            future_data = pd.DataFrame({
                'city': [city],
                'year': [year]
            })

            # Transform and normalize the future data
            future_data_transformed = column_transformer_first.transform(future_data)
            future_data_transformed_normalized = scaler_X_first.transform(future_data_transformed)
            future_data_transformed_normalized = np.array(future_data_transformed_normalized).reshape(-1, 1, future_data_transformed_normalized.shape[1])

            # Make predictions for the future year
            future_predictions = model_first.predict(future_data_transformed_normalized)
            future_predictions_inverse = scaler_Y_first.inverse_transform(future_predictions)

            # Extract predicted values
            predicted_income, predicted_mean_age, predicted_unemployment, predicted_truck_price, predicted_geomobility = future_predictions_inverse[0]

            # Prepare the dataset for the second regression
            future_housing_data = pd.DataFrame({
                'city': [city],
                'year': [year],
                'income': [predicted_income],
                'median_age': [predicted_mean_age],
                'unemployment': [predicted_unemployment],
                'truck_cost': [predicted_truck_price],
                'geomobility': [predicted_geomobility]
            })

            # Transform and normalize the future housing data
            future_housing_data_transformed = column_transformer_second.transform(future_housing_data)
            future_housing_data_transformed_normalized = scaler_X_second.transform(future_housing_data_transformed)
            future_housing_data_transformed_normalized = np.array(future_housing_data_transformed_normalized).reshape(-1, 1, future_housing_data_transformed_normalized.shape[1])

            # Make predictions for housing price
            predicted_housing_price = model_second.predict(future_housing_data_transformed_normalized)
            predicted_housing_price_inverse = scaler_Y_second.inverse_transform(predicted_housing_price)

            # Store the results
            results.append({
                'city': city,
                'year': year,
                'predicted_income': predicted_income,
                'predicted_mean_age': predicted_mean_age,
                'predicted_unemployment': predicted_unemployment,
                'predicted_truck_cost': predicted_truck_price,
                'predicted_geomobility': predicted_geomobility,
                'predicted_housing_price': predicted_housing_price_inverse[0][0]
            })

    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

data = pd.read_csv("../data/merged_data.csv")
data = data.drop("Unnamed: 0", axis=1)
# Train models only once
model_first, scaler_X_first, scaler_Y_first, column_transformer_first, model_second, scaler_X_second, scaler_Y_second, column_transformer_second = train_models(data)

# Example usage
cities = data["city"].unique()  # List of unique cities
years = [2024, 2025, 2026]  # List of years
predictions_df = predict_city_data(cities, years, model_first, scaler_X_first, scaler_Y_first, column_transformer_first, model_second, scaler_X_second, scaler_Y_second, column_transformer_second)

predictions_df


Epoch 1/300


/Users/achyutannarayanan/Desktop/College/HackGT/Realty_Pulse/djangoProject/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1422 - val_loss: 0.1570
Epoch 2/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1172 - val_loss: 0.1201
Epoch 3/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0819 - val_loss: 0.0723
Epoch 4/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0435 - val_loss: 0.0425
Epoch 5/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0313 - val_loss: 0.0413
Epoch 6/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0273 - val_loss: 0.0400
Epoch 7/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0242 - val_loss: 0.0392
Epoch 8/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0228 - val_loss: 0.0384
Epoch 9/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0201 - val_loss: 0.0378
Epoch 10/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0176 - val_loss: 0.0369
Epoch 11/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0156 - val_loss: 0.0370
Epoch 12/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0135 - val_lo

/Users/achyutannarayanan/Desktop/College/HackGT/Realty_Pulse/djangoProject/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0628 - val_loss: 0.0994
Epoch 2/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0369 - val_loss: 0.0523
Epoch 3/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0172 - val_loss: 0.0329
Epoch 4/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0137 - val_loss: 0.0336
Epoch 5/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0089 - val_loss: 0.0268
Epoch 6/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0060 - val_loss: 0.0236
Epoch 7/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0042 - val_loss: 0.0201
Epoch 8/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0032 - val_loss: 0.0194
Epoch 9/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0027 - val_loss: 0.0180
Epoch 10/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0030 - val_loss: 0.0178
Epoch 11/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0024 - val_loss: 0.0174
Epoch 12/300
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0025 - val_lo

,city,year,predicted_income,predicted_mean_age,predicted_unemployment,predicted_truck_cost,predicted_geomobility,predicted_housing_price
0,"Albuquerque, NM",2024,70163.093750,38.948875,4.580811,1959.479004,529106.87500,330278.34375
1,"Albuquerque, NM",2025,73544.835938,39.259460,4.533796,1966.566895,527790.81250,360997.65625
2,"Albuquerque, NM",2026,77032.039062,39.585419,4.471187,1976.406006,526923.43750,394179.53125
3,"Atlanta, GA",2024,87295.843750,34.588352,4.297268,1905.922974,515154.25000,389576.31250
4,"Atlanta, GA",2025,91836.054688,34.905041,4.124164,1923.769287,525319.87500,431053.62500
...,...,...,...,...,...,...,...,...
94,"Washington, DC",2025,69676.054688,40.104797,5.263146,2085.445068,515495.93750,401828.62500
95,"Washington, DC",2026,73037.820312,40.452381,5.177754,2095.768066,516473.75000,436067.59375
96,"Wichita, KS",2024,67480.460938,39.294098,4.588482,2082.765137,377563.00000,338854.43750
97,"Wichita, KS",2025,70770.648438,39.738792,4.540356,2095.233887,368996.84375,366178.50000
